# Parallel execution tree expansion with the simulator API

Questo notebook mostra come usare le API REST del simulatore per generare un execution tree in parallelo. L'obiettivo è mantenere l'albero condiviso tra i thread, delegando la creazione dei nodi al simulatore e rispettando il vincolo che ogni thread che genera figli con una natura attiva continui immediatamente l'espansione di quella sottostruttura.


## Prerequisiti

* Il simulatore deve essere in esecuzione e raggiungibile tramite HTTP (porta predefinita `8001`).
* È necessario avere installate le dipendenze Python del progetto PACO (in particolare `requests` e i moduli presenti nella cartella `src/`).
* Il notebook non modifica i sorgenti esistenti: tutto il codice necessario (client HTTP, scheduler dei thread e funzioni di utilità) è definito qui.


In [ ]:
import json
import os
import sys
import threading
import queue
import itertools
from dataclasses import dataclass
from typing import Any, Dict, Iterable, List, Optional, Tuple

import requests

# Rendi disponibili i moduli di PACO
PROJECT_SRC = os.path.join(os.getcwd(), "src")
if PROJECT_SRC not in sys.path:
    sys.path.append(PROJECT_SRC)

from paco.parser.bpmn_parser import create_parse_tree
from paco.parser.parse_tree import ParseTree
from utils.env import EXPRESSION, IMPACTS, DURATIONS, DELAYS, PROBABILITIES, LOOP_PROBABILITY, LOOP_ROUND, H, IMPACTS_NAMES

SIMULATOR_SERVER = os.getenv("SIMULATOR_SERVER", "http://127.0.0.1:8001/")
HEADERS = {"Content-Type": "application/json"}


## BPMN di esempio

Per dimostrare l'algoritmo usiamo un semplice processo con tre attività (`Cutting`, `Bending`, `Milling`) dove `Cutting` e `Bending` possono eseguire in parallelo. Gli impatti e le durate sono fittizi e servono solamente a popolare il parse tree.


In [ ]:
example_bpmn = {
    EXPRESSION: "(Cutting||Bending),Milling",
    IMPACTS: {
        "Cutting": [10.0, 1.0],
        "Bending": [20.0, 1.5],
        "Milling": [50.0, 2.0],
    },
    DURATIONS: {
        "Cutting": [0, 1],
        "Bending": [0, 1],
        "Milling": [0, 1],
    },
    PROBABILITIES: {},
    DELAYS: {},
    LOOP_PROBABILITY: {},
    LOOP_ROUND: {},
    H: 0,
    IMPACTS_NAMES: ["energy", "hours"],
}

parse_tree, pending_choices, pending_natures, pending_loops = create_parse_tree(example_bpmn)
parse_tree_dict = parse_tree.to_dict()
print("Parse tree generato con", len(pending_choices), "scelte pendenti e", len(pending_natures), "nature pendenti.")


## Client HTTP verso il simulatore

Il `SimulatorClient` incapsula le chiamate ai due endpoint documentati (`/execute` e `/steps`). Entrambe le richieste accettano un payload JSON e restituiscono strutture pronte da reinserire nell'albero condiviso.


In [ ]:
class SimulatorClient:
    """Wrapper minimale per le chiamate REST del simulatore."""

    def __init__(self, base_url: str = SIMULATOR_SERVER, session: Optional[requests.Session] = None) -> None:
        self.base_url = base_url.rstrip("/") + "/"
        self.session = session or requests.Session()

    def execute(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """Esegue una POST su /execute restituendo il JSON della risposta."""
        response = self.session.post(self.base_url + "execute", headers=HEADERS, json=payload, timeout=60)
        response.raise_for_status()
        data = response.json()
        if "error" in data:
            raise RuntimeError(f"Simulator error: {data['error']}")
        return data

    def steps(self, payload: Dict[str, Any]) -> Dict[str, Any]:
        """Esegue una POST su /steps restituendo il JSON della risposta."""
        response = self.session.post(self.base_url + "steps", headers=HEADERS, json=payload, timeout=60)
        response.raise_for_status()
        data = response.json()
        if "error" in data:
            raise RuntimeError(f"Simulator error: {data['error']}")
        return data

    def bootstrap(self, parse_tree_dict: Dict[str, Any]) -> Dict[str, Any]:
        """Richiede al simulatore l'inizializzazione partendo dal parse tree."""
        payload = {"bpmn": parse_tree_dict}
        return self.execute(payload)

    def expand_node(
        self,
        shared_state: Dict[str, Any],
        node_id: int,
        *,
        choices: Optional[Iterable[str]] = None,
    ) -> Dict[str, Any]:
        """Chiede al simulatore di espandere un nodo già presente nell'albero."""
        request_payload = {
            "bpmn": shared_state["bpmn"],
            "petri_net": shared_state["petri_net"],
            "petri_net_dot": shared_state["petri_net_dot"],
            "execution_tree": shared_state["execution_tree"],
            "node_id": node_id,
        }
        if choices is not None:
            request_payload["choices"] = list(choices)
        response = self.steps(request_payload)
        if "node" not in response:
            raise KeyError("Simulator response does not contain the expanded node JSON")
        return response


## Utilità per navigare l'albero

Funzioni di supporto per contare i nodi, estrarre percorsi e aggiornare porzioni dell'albero senza blocchi globali.


In [ ]:
def iter_pending_paths(node: Dict[str, Any], path: Tuple[str, ...] = ()) -> Iterable[Tuple[str, ...]]:
    """Restituisce tutti i percorsi verso nodi con scelte o nature pendenti."""
    if node.get("pending_choices") or node.get("pending_natures"):
        yield path
    for key, child in node.get("transitions", {}).items():
        yield from iter_pending_paths(child, path + (key,))


def get_node_at_path(root: Dict[str, Any], path: Tuple[str, ...]) -> Dict[str, Any]:
    """Segue il percorso e restituisce il dizionario del nodo corrispondente."""
    node = root
    for key in path:
        node = node["transitions"][key]
    return node


def set_node_at_path(root: Dict[str, Any], path: Tuple[str, ...], new_node: Dict[str, Any]) -> None:
    """Sostituisce un nodo all'interno dell'albero seguendo il percorso dato."""
    if not path:
        root.clear()
        root.update(new_node)
        return
    parent = get_node_at_path(root, path[:-1])
    parent["transitions"][path[-1]] = new_node


def count_nodes(node: Dict[str, Any]) -> int:
    """Conta ricorsivamente i nodi dell'albero."""
    total = 1
    for child in node.get("transitions", {}).values():
        total += count_nodes(child)
    return total


## Scheduler parallelo

`ParallelExecutionTreeBuilder` coordina i thread e garantisce che i nodi figli con nature attive vengano espansi immediatamente dallo stesso worker. Ogni nodo dell'albero possiede un proprio lock per evitare un collo di bottiglia globale durante l'inserimento dei figli.


In [ ]:
class ParallelExecutionTreeBuilder:
    """Espande un execution tree sfruttando il simulatore in parallelo."""

    def __init__(self, client: SimulatorClient, max_workers: int = 1) -> None:
        if max_workers < 1:
            raise ValueError("max_workers must be >= 1")
        self.client = client
        self.max_workers = max_workers
        self._queue: "queue.Queue[Tuple[str, ...]]" = queue.Queue()
        self._node_locks: Dict[int, threading.Lock] = {}
        self._locks_guard = threading.Lock()
        self._stop_event = threading.Event()

    def _lock_for(self, node_id: int) -> threading.Lock:
        with self._locks_guard:
            lock = self._node_locks.get(node_id)
            if lock is None:
                lock = threading.Lock()
                self._node_locks[node_id] = lock
            return lock

    def _enqueue_initial_frontier(self, tree_root: Dict[str, Any]) -> None:
        for path in iter_pending_paths(tree_root):
            self._queue.put(path)

    def _schedule_children(self, node: Dict[str, Any], path: Tuple[str, ...]) -> None:
        sequential_expansions: List[Tuple[str, ...]] = []
        for key, child in node.get("transitions", {}).items():
            child_path = path + (key,)
            if child.get("pending_choices") or child.get("pending_natures"):
                if child.get("natures"):
                    sequential_expansions.append(child_path)
                else:
                    self._queue.put(child_path)
        for child_path in sequential_expansions:
            self._expand_path(child_path)

    def _expand_path(self, path: Tuple[str, ...]) -> None:
        if self._stop_event.is_set():
            return
        node = get_node_at_path(self._shared_tree["execution_tree"]["root"], path)
        if not (node.get("pending_choices") or node.get("pending_natures")):
            return
        node_id = node["id"]
        lock = self._lock_for(node_id)
        with lock:
            response = self.client.expand_node(self._shared_tree, node_id)
            expanded_node = response["node"]
            set_node_at_path(self._shared_tree["execution_tree"]["root"], path, expanded_node)
            self._shared_tree["execution_tree"] = response.get("execution_tree", self._shared_tree["execution_tree"])
        self._schedule_children(expanded_node, path)

    def _worker(self) -> None:
        while not self._stop_event.is_set():
            try:
                path = self._queue.get(timeout=0.1)
            except queue.Empty:
                if self._queue.empty():
                    break
                continue
            try:
                self._expand_path(path)
            finally:
                self._queue.task_done()

    def build(self, parse_tree_dict: Dict[str, Any]) -> Dict[str, Any]:
        bootstrap = self.client.bootstrap(parse_tree_dict)
        self._shared_tree = {
            "bpmn": bootstrap["bpmn"],
            "petri_net": bootstrap["petri_net"],
            "petri_net_dot": bootstrap["petri_net_dot"],
            "execution_tree": bootstrap["execution_tree"],
        }
        self._enqueue_initial_frontier(self._shared_tree["execution_tree"]["root"])
        workers: List[threading.Thread] = []
        for idx in range(self.max_workers):
            t = threading.Thread(target=self._worker, name=f"tree-worker-{idx}", daemon=True)
            t.start()
            workers.append(t)
        self._queue.join()
        self._stop_event.set()
        for t in workers:
            t.join()
        return self._shared_tree


## Esecuzione della costruzione parallela

L'esempio seguente utilizza quattro thread. Alla fine viene mostrato il numero totale di nodi generati.


In [ ]:
client = SimulatorClient()
builder = ParallelExecutionTreeBuilder(client, max_workers=4)

shared_state = builder.build(parse_tree_dict)
root_node = shared_state["execution_tree"]["root"]
print("Numero totale di nodi generati:", count_nodes(root_node))


## Analisi opzionale dell'albero

Se si desidera utilizzare le classi di PACO per ulteriori analisi (ad esempio calcolare impatti o esportare in DOT), è sufficiente convertire il JSON restituito dal simulatore in un `ExecutionTree`.


In [ ]:
from paco.execution_tree.execution_tree import ExecutionTree

parse_tree_obj, *_ = ParseTree.from_json(parse_tree_dict)
execution_tree = ExecutionTree.from_json(parse_tree_obj, shared_state["execution_tree"], example_bpmn[IMPACTS_NAMES])
print("Radice dell'execution tree:", execution_tree.root.id)
